<a href="https://colab.research.google.com/github/SeanMuInCa/learn_python/blob/master/Assignment1%20Naive%20Bayes%20Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This assignment done by Zhenghua Mu

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shivadumnawar/titanic-dataset")

print("Path to dataset files:", path)

100%|██████████| 33.1k/33.1k [00:00<00:00, 34.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/shivadumnawar/titanic-dataset/versions/1


In [ ]:
import pandas as pd
# load the data
trainDF = pd.read_csv(path + "/train.csv")
trainDF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# features select
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

# get data
X = trainDF[features]
y = trainDF['Survived']


X.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [ ]:

X = X.copy()  # avoid warning

# deal with missing data
X['Age'] = X['Age'].fillna(X['Age'].mean())
X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])

# check missing data
X.isnull().sum()



,0
Pclass,0
Sex,0
Age,0
SibSp,0
Parch,0
Fare,0
Embarked,0


In [ ]:

X = pd.get_dummies(X, columns=['Sex', 'Embarked'], drop_first=True)

X.head()


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,True,False,True
1,1,38.0,1,0,71.2833,False,False,False
2,3,26.0,0,0,7.9250,False,False,True
3,1,35.0,1,0,53.1000,False,False,True
4,3,35.0,0,0,8.0500,True,False,True


In [ ]:
from sklearn.model_selection import train_test_split

# split data as test 30 70
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((623, 8), (268, 8), (623,), (268,))

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

# init model
model = GaussianNB()

# train
model.fit(X_train, y_train)

# cross_val_score train model
cv_scores = cross_val_score(model, X_train, y_train, cv=5)

# scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())


Cross-validation scores: [0.792      0.84       0.76       0.75       0.78225806]
Mean cross-validation score: 0.7848516129032259


In [ ]:
# test accuracy
test_accuracy = model.score(X_test, y_test)


print("Test set accuracy:", test_accuracy)


Test set accuracy: 0.7947761194029851


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# SVM model
svm = SVC()

# hyperparams
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.001],
    'kernel': ['rbf']
}


# search the best params set
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)


print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters found:  {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Best cross-validation score:  0.7447870967741935


In [ ]:
# use the best match params
best_svm = SVC(C=10, gamma=0.01, kernel='rbf')

# train
best_svm.fit(X_train, y_train)

# test accuracy
svm_test_accuracy = best_svm.score(X_test, y_test)


print("SVM Test set accuracy:", svm_test_accuracy)


SVM Test set accuracy: 0.753731343283582


In [ ]:
from sklearn.metrics import classification_report

# predict test result
nb_predictions = model.predict(X_test)
svm_predictions = best_svm.predict(X_test)

# print reports
print("Naive Bayes Classification Report:")
print(classification_report(y_test, nb_predictions))

print("SVM Classification Report:")
print(classification_report(y_test, svm_predictions))


Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       157
           1       0.75      0.75      0.75       111

    accuracy                           0.79       268
   macro avg       0.79      0.79      0.79       268
weighted avg       0.79      0.79      0.79       268

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.85      0.80       157
           1       0.75      0.61      0.67       111

    accuracy                           0.75       268
   macro avg       0.75      0.73      0.74       268
weighted avg       0.75      0.75      0.75       268



Conclusion:

Accuracy: The Naive Bayes classifier performs better in terms of overall accuracy.

Recall: The Naive Bayes classifier has a more balanced recall across both categories, the SVM model has a lower recall for category 1.

F1 Score: The Naive Bayes classifier has higher F1 scores for both categories, indicating a better balance between precision and recall.

Naive Bayes Classifier performs better on this dataset, probably because its assumption of feature independence is a reasonable approximation for this dataset.

Although SVM Model has a higher recall for category 0, its performance is weak for category 1, that means hyperparameters tuning is necessary on this situation.